Création d'un dataframe reprenant les deux fichiers de logs

In [64]:
import pandas as pd
import matplotlib.pyplot as plt

In [65]:
df_err = pd.read_csv('../Data/logEtl/241016_LogETLError.csv', sep=';', dtype={'Program_Id': str, 'Schedules_Id': str, 'Schedules_Name': str})

In [66]:
df = pd.read_csv('../data/logEtl/241016_LogETL.csv',sep=';')

In [67]:
df.columns

Index(['Server version', 'Client version', 'Model', 'Type', 'Insert mode',
       'Rows added', 'Rows updated', 'Rows deleted', 'Rows in error',
       'Rows in warning', 'Columns', 'Date', 'Start time', 'End  time',
       'Duration', 'Machine', 'Session', 'Project name', 'Product', 'Result',
       'ETL_StartDateTime', 'Launcher_Id', 'Launcher_Name', 'Program_Id',
       'Program_Name', 'Schedules_Id', 'Schedules_Name',
       'Schedules_StartDateTime'],
      dtype='object')

In [68]:
df_filtered = df.drop(columns=['Server version', 'Client version'])

In [69]:
df_filtered.columns

Index(['Model', 'Type', 'Insert mode', 'Rows added', 'Rows updated',
       'Rows deleted', 'Rows in error', 'Rows in warning', 'Columns', 'Date',
       'Start time', 'End  time', 'Duration', 'Machine', 'Session',
       'Project name', 'Product', 'Result', 'ETL_StartDateTime', 'Launcher_Id',
       'Launcher_Name', 'Program_Id', 'Program_Name', 'Schedules_Id',
       'Schedules_Name', 'Schedules_StartDateTime'],
      dtype='object')

In [70]:
df_err.columns

Index(['Server version', 'Client version', 'Product', 'Project name', 'Model',
       'Date', 'Time', 'Row number', 'Type', 'Message', 'ETL_StartDateTime',
       'Launcher_Id', 'Launcher_Name', 'Machine', 'Program_Id', 'Program_Name',
       'Schedules_Id', 'Schedules_Name', 'Schedules_StartDateTime'],
      dtype='object')

In [71]:
df_err_filtered = df_err.drop(columns=['Server version', 'Client version'])

In [45]:
#df_err_filtered.columns = [col + "_err" for col in df_err_filtered.columns]

In [72]:
df_err_filtered.columns

Index(['Product', 'Project name', 'Model', 'Date', 'Time', 'Row number',
       'Type', 'Message', 'ETL_StartDateTime', 'Launcher_Id', 'Launcher_Name',
       'Machine', 'Program_Id', 'Program_Name', 'Schedules_Id',
       'Schedules_Name', 'Schedules_StartDateTime'],
      dtype='object')

In [74]:
def categorize_message(message):
    message = message.lower()
    if "Guidez Atelier".lower() in message or "Guidez Social".lower()  in message:
        return "Guidez Atelier Error"
    elif "La taille du champ".lower()  in message:
        return "Field Size Error" 
    elif "Un incident est survenu Object reference not set to an instance of an object.".lower()  in message:
        return "Object Reference Error"
    elif "La valeur #REF!".lower()  in message:
        return "Value #REF! Error"
    elif "La valeur #VALUE!".lower()  in message:
        return "Value #VALUE! Error"
    elif "La valeur #N/A!".lower()  in message:
        return "Value #N/A! Error"
    elif "La valeur #DIV/0!".lower()  in message:
        return "Value #DIV/0! Error"
    elif "La valeur #NAME?".lower()  in message:
        return "Value #NAME? Error"
    elif "Connection Timeout Expired.  The timeout period elapsed during the post-login phase.".lower()  in message:
        return "Connection Timeout Error"
    elif "an error occurred during the pre-login handshake.".lower()  in message:
        return "Pre-Login Error"
    elif "Un incident est survenu L'instruction CREATE UNIQUE INDEX a été interrompue, car une clé dupliquée a été trouvée pour l'objet".lower()  in message:
        return "Unique Index Error"
    elif "Impossible de créer la table".lower()  in message:
        return "Create TABLE Error"
    elif "Un incident est survenu Échec de l'opération car un index ou des statistiques portant le nom".lower()  in message:
        return "Index Error (similar to unique index error)"
    elif "utilisé dans la clé primaire, sa valeur ne doit pas être vide".lower()  in message:
        return "Primary Key Error"
    elif "pas convertible en Heure".lower()  in message:
        return "Time Format Error"
    elif "Not a legal OleAut date.".lower()  in message or ("La date".lower()  in message and "est pas valide".lower()  in message):
        return "Date Format Error"
    elif "connexion au web service impossible".lower()  in message:
        return "Web Service Error"
    elif "impossible d'ouvrir la requête".lower()  in message and "Un incident est survenu".lower()  in message and "non valide".lower()  in message:
        return "SQL Invalid Query Error"
    elif "Impossible d\'ouvrir la requête SQL sur la connexion".lower()  in message and "Le délai d\'attente a été dépassé".lower()  in message:
        return "SQL Query Timeout Error"
    #elif "Le filtre de suppression n'est pas compatible SQL".lower()  in message:
    #    return "SQL Delete Filter Error"
    #elif "impossible d'ouvrir la requête sql".lower()  in message:
     #   return "SQL Other Error"
    elif "Le nom de colonne".lower()  in message and "n'existe pas dans la table ou la vue cible".lower()  in message:
        return "Column Name Error"
    else:
        return "Other Error"

df_err_filtered = df_err_filtered.assign(
    Message_Category=df_err_filtered["Message"].apply(categorize_message)
)

In [75]:
colonnes_communes = list(set(df_filtered.columns) & set(df_err_filtered.columns))
print(colonnes_communes)

['Schedules_Name', 'Model', 'Product', 'ETL_StartDateTime', 'Launcher_Name', 'Machine', 'Program_Name', 'Date', 'Launcher_Id', 'Schedules_Id', 'Schedules_StartDateTime', 'Project name', 'Type', 'Program_Id']


In [76]:
# Ajouter les colonnes manquantes dans chaque DataFrame pour les aligner
for col in df_filtered.columns:
    if col not in df_err_filtered.columns:
        df_err_filtered[col] = None  # Ajout d'une colonne vide

for col in df_err_filtered.columns:
    if col not in df_filtered.columns:
        df_filtered[col] = None  # Ajout d'une colonne vide

# Concaténer les deux DataFrame tout en conservant la cohérence des colonnes
df_final = pd.concat([df_filtered, df_err_filtered], ignore_index=True)

df_final.head()  # Affiche les 5 premières lignes

,Model,Type,Insert mode,Rows added,Rows updated,Rows deleted,Rows in error,Rows in warning,Columns,Date,...,Launcher_Name,Program_Id,Program_Name,Schedules_Id,Schedules_Name,Schedules_StartDateTime,Time,Row number,Message,Message_Category
0,TMS/MF_TrajetsNew,Modèle simple,Mise à jour données / Insertion données,36861,0,36642,NaN,NaN,49,2024-03-14,...,Serveur (programmation),ETL_b75838e4-8850-4d9a-ba25-4d541f4b237f,ETL Opti TMS 1h,ETLPROGRAMMATIONBG7G6W,ETL Opti TMS toutes les 1h Lun-Sam,2024-03-14 15:41:46,None,None,None,None
1,TMS/MF_FactureFournisseursSansDetails,Modèle simple,Mise à jour données / Insertion données,118,0,44,NaN,NaN,34,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
2,TMS/MF_FacturesSansDetails,Modèle simple,Mise à jour données / Insertion données,369,0,17,NaN,NaN,37,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
3,TMS/MF_SAV,Modèle simple,Mise à jour données / Insertion données,277,0,191,NaN,NaN,75,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
4,TMS/MF_FactureFournisseurs,Modèle simple,Mise à jour données / Insertion données,1036,0,428,NaN,NaN,61,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None


In [77]:
# Assurer que ETL_StartDateTime est bien au format datetime
df_final["ETL_StartDateTime"] = pd.to_datetime(df_final["ETL_StartDateTime"], errors='coerce')

# Trier le DataFrame par ETL_StartDateTime
df_final = df_final.sort_values(by="ETL_StartDateTime")

# Afficher les premières lignes pour vérifier
df_final.head()

,Model,Type,Insert mode,Rows added,Rows updated,Rows deleted,Rows in error,Rows in warning,Columns,Date,...,Launcher_Name,Program_Id,Program_Name,Schedules_Id,Schedules_Name,Schedules_StartDateTime,Time,Row number,Message,Message_Category
0,TMS/MF_TrajetsNew,Modèle simple,Mise à jour données / Insertion données,36861,0,36642,NaN,NaN,49,2024-03-14,...,Serveur (programmation),ETL_b75838e4-8850-4d9a-ba25-4d541f4b237f,ETL Opti TMS 1h,ETLPROGRAMMATIONBG7G6W,ETL Opti TMS toutes les 1h Lun-Sam,2024-03-14 15:41:46,None,None,None,None
1,TMS/MF_FactureFournisseursSansDetails,Modèle simple,Mise à jour données / Insertion données,118,0,44,NaN,NaN,34,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
2,TMS/MF_FacturesSansDetails,Modèle simple,Mise à jour données / Insertion données,369,0,17,NaN,NaN,37,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
3,TMS/MF_SAV,Modèle simple,Mise à jour données / Insertion données,277,0,191,NaN,NaN,75,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None
4,TMS/MF_FactureFournisseurs,Modèle simple,Mise à jour données / Insertion données,1036,0,428,NaN,NaN,61,2024-03-14,...,Serveur (programmation),ETL_9fdd54e0-2573-4307-af52-efe1c4c6a0c0,ETL Opti TMS Jour,ETLPROGRAMMATIONLFMOU5,ETL Opti TMS-Compta 13h-16h Lun-Sam,2024-03-14 16:01:38,None,None,None,None


In [78]:
# Définir les bornes
date_min = "2024-08-24 23:32:03"
date_max = "2024-10-15 23:31:49"

# Filtrer les lignes entre les deux dates
df_borne = df_final[(df_final["ETL_StartDateTime"] >= date_min) & (df_final["ETL_StartDateTime"] <= date_max)]

# Afficher les premières lignes pour vérifier
df_borne.head()

,Model,Type,Insert mode,Rows added,Rows updated,Rows deleted,Rows in error,Rows in warning,Columns,Date,...,Launcher_Name,Program_Id,Program_Name,Schedules_Id,Schedules_Name,Schedules_StartDateTime,Time,Row number,Message,Message_Category
122704,TMS/MF_GrilleValorisation,Modèle simple,Suppression table / Création table / Insertion...,92,0,0,NaN,NaN,10,2024-08-25,...,Serveur (programmation),ETL_3a685d15-3172-4faf-92e5-4b0918f6b315,Tous les jours,ETLPROGRAMMATIONCYZ21D,ETL Jour 23h30,2024-08-24 23:31:51,None,None,None,None
122705,TMS/MF_GroupeTiers,Modèle simple,Suppression table / Création table / Insertion...,14624,0,0,NaN,NaN,7,2024-08-25,...,Serveur (programmation),ETL_3a685d15-3172-4faf-92e5-4b0918f6b315,Tous les jours,ETLPROGRAMMATIONCYZ21D,ETL Jour 23h30,2024-08-24 23:31:51,None,None,None,None
122706,TMS/MF_HistoDetail,Modèle simple,Suppression table / Création table / Insertion...,5480996,0,0,NaN,NaN,5,2024-08-25,...,Serveur (programmation),ETL_3a685d15-3172-4faf-92e5-4b0918f6b315,Tous les jours,ETLPROGRAMMATIONCYZ21D,ETL Jour 23h30,2024-08-24 23:31:51,None,None,None,None
122707,TMS/MF_GED_Exploit,Modèle simple,Suppression table / Création table / Insertion...,9128644,0,0,NaN,NaN,9,2024-08-25,...,Serveur (programmation),ETL_3a685d15-3172-4faf-92e5-4b0918f6b315,Tous les jours,ETLPROGRAMMATIONCYZ21D,ETL Jour 23h30,2024-08-24 23:31:51,None,None,None,None
122637,Excel MyReport/Commerce/PrimForce/PROJETS/Prim...,Modèle Excel,Suppression table / Création table / Insertion...,151,0,0,NaN,NaN,51,2024-08-25,...,Serveur (programmation),ETL_3a685d15-3172-4faf-92e5-4b0918f6b315,Tous les jours,ETLPROGRAMMATIONCYZ21D,ETL Jour 23h30,2024-08-24 23:31:51,None,None,None,None


In [79]:
df_borne.to_csv("fusion_dataframe.csv", index=False)
print("Fichier exporté : fusion_dataframe.csv")

Fichier exporté : fusion_dataframe.csv


In [80]:
df_borne.columns

Index(['Model', 'Type', 'Insert mode', 'Rows added', 'Rows updated',
       'Rows deleted', 'Rows in error', 'Rows in warning', 'Columns', 'Date',
       'Start time', 'End  time', 'Duration', 'Machine', 'Session',
       'Project name', 'Product', 'Result', 'ETL_StartDateTime', 'Launcher_Id',
       'Launcher_Name', 'Program_Id', 'Program_Name', 'Schedules_Id',
       'Schedules_Name', 'Schedules_StartDateTime', 'Time', 'Row number',
       'Message', 'Message_Category'],
      dtype='object')